In [1]:
from PIL import Image, ImageDraw, ImageFont

In [3]:
def generate_cropped_letter_image(letter='A', font_path='arial.ttf', font_size=48):
    """
    1) 在一张较大(100×100)的 '1' 模式(纯黑白)图像中，居中绘制字母(无抗锯齿)；
    2) 使用 textbbox 计算文字范围，并裁剪多余空白。
    返回裁剪后的纯黑白 PIL.Image。
    """
    from PIL import Image, ImageDraw, ImageFont
    
    big_size = 200
    # 1-bit模式，初始全白 (fill=1)
    img = Image.new('1', (big_size, big_size), 1)
    draw = ImageDraw.Draw(img)

    font = ImageFont.truetype(font_path, font_size)

    # 使用 textbbox 获取绘制宽高
    # 注意：textbbox 需要先有参照点，这里可以先写 (0,0)
    bbox = draw.textbbox((0,0), letter, font=font)
    text_w = bbox[2] - bbox[0]
    text_h = bbox[3] - bbox[1]

    # 让文字居中
    x = (big_size - text_w) // 2
    y = (big_size - text_h) // 2

    # 在 (x,y) 绘制黑色字母
    draw.text((x, y), letter, font=font, fill=0)

    # 重新用 textbbox 获取真正的绘制范围
    # 注意：这次要用 (x,y) 作为参照
    bbox = draw.textbbox((x, y), letter, font=font)
    left, top, right, bottom = bbox  # left, top, right, bottom

    # 若整幅图都是空白(可能用户传入空字符)
    if left == right or top == bottom:
        return img  # 或者返回一个空图

    # 适当留一些 margin，避免切得太紧
    margin = 1
    left   = max(left - margin, 0)
    top    = max(top - margin, 0)
    right  = min(right + margin, big_size)
    bottom = min(bottom + margin, big_size)

    cropped = img.crop((left, top, right, bottom))
    return cropped

In [4]:
def image_to_nonogram(img, final_size=(10,10)):
    """
    将 1-bit 图像(或其他模式图像) 转换为 Nonogram 的(0/1)矩阵 + 行列提示。
    流程：
      - 缩放到 final_size (NEAREST)
      - 读取像素，0=黑, 1=白 => or 反过来
      - 生成行/列提示
    返回 (matrix_2d, row_constraints, col_constraints)
    """
    # 1) 缩放到指定尺寸
    resized = img.resize(final_size, Image.NEAREST)

    # 2) 读取像素 => 0/1 矩阵
    pixel_data = resized.load()
    height, width = final_size[1], final_size[0]
    matrix_2d = []
    for y in range(height):
        row = []
        for x in range(width):
            # 在 '1' 模式下，pixel_data[x,y] 要么0(黑)要么1(白)
            val = pixel_data[x, y]
            # 这里先写 1=黑, 0=白
            row.append(1 - val)  # val=0 => black => 1; val=1 => white => 0
        matrix_2d.append(row)

    # 3) 生成行列提示
    row_constraints = []
    for r in range(height):
        hint = []
        count = 0
        for c in range(width):
            if matrix_2d[r][c] == 1:
                count += 1
            else:
                if count > 0:
                    hint.append(count)
                    count = 0
        if count > 0:
            hint.append(count)
        row_constraints.append(hint)

    col_constraints = []
    for c in range(width):
        hint = []
        count = 0
        for r in range(height):
            if matrix_2d[r][c] == 1:
                count += 1
            else:
                if count > 0:
                    hint.append(count)
                    count = 0
        if count > 0:
            hint.append(count)
        col_constraints.append(hint)

    return matrix_2d, row_constraints, col_constraints

In [7]:
def letter_to_nonogram(letter='A', font_path='arial.ttf', font_size=48,
                       final_size=(10,10)):
    """
    一步到位：先用 1-bit 模式生成(裁剪)字母图，无抗锯齿，
    再转换到 Nonogram 矩阵与提示。
    返回 (matrix_2d, row_constraints, col_constraints)
    """
    # 1) 生成并裁剪字母图 (纯黑白)
    cropped_img = generate_cropped_letter_image(letter, font_path, font_size)
    
    # 2) 转换到 Nonogram
    matrix_2d, row_cons, col_cons = image_to_nonogram(
        cropped_img, final_size=final_size
    )
    return matrix_2d, row_cons, col_cons

In [92]:
if __name__ == "__main__":
    # 测试: 生成 'A'，用默认 'arial.ttf' 字体，12号，缩放到(12,12)
    matrix, rows, cols = letter_to_nonogram(
        letter="A",
        font_path='arial.ttf',
        font_size=48,
        final_size=(20,20)
    )

    # 1) 先打印普通数值矩阵
    print("Matrix (0=white, 1=black):")
    for row in matrix:
        print(" ".join(str(x) for x in row))

    print("\nrow_constraints =", rows)
    print("col_constraints =", cols)

    # 2) 再打印带“黑白方块”样式
    print("\n---- ASCII黑白格子样式 ----")
    for row in matrix:
        # 如果为1就显示"■"，如果是0就显示" "（空格）
        line = "".join("■" if x == 1 else " " for x in row)
        print(line)

Matrix (0=white, 1=black):
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 1 1 0 1 1 0 0 0 0 0 0 0 0
0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0
0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0
0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0
0 0 0 0 0 1 1 0 0 0 0 1 1 1 0 0 0 0 0 0
0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 0 0 0
0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0
0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0
0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0
0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0
0 0 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0
0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0
0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0
0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

row_constraints = [[], [3], [4], [2, 1], [2, 2], [2, 2], [2, 2], [1, 2], [2, 3], [2, 2], [2, 2], [11], [12], [13], [2, 2], [3, 3], [2, 2], [2, 3], [2, 3], []]
col_constrain